# Digit Recognition App

To run: Kernel -> Restart & Run All

In [1]:
import ipywidgets as w
import pandas as pd

from SimpleCV import *
from ipywidgets import widgets, HBox, VBox
from IPython.display import display
from sklearn.externals import joblib

disp = Display(displaytype='notebook')
app = w.Output()

FILES = []

SIZE = 32 # SINGLE DIGIT IMAGE SIZE
COUNT = 8 # GRID SIZE

### Importing Normalizer & Classifier

We also need the columns the classifier was trained on and the training set was normalized on.

In [2]:
CLASSIFIER = joblib.load('../classifiers/classifier_svm_v8.joblib')
SCALER = joblib.load('../classifiers/scaler_svm_v8.joblib')
COLUMNS = [u'area', u'width', u'contours', u'radius', u'circle_dist', u'rect_dist',
       u'hull_radius', u'aspect_ratio', u'centroid_x', u'centroid_y',
       u'corners', u'circles', u'angle', u'weight_0_0', u'weight_0_1',
       u'weight_0_2', u'weight_0_3', u'weight_0_4', u'weight_0_5',
       u'weight_0_6', u'weight_0_7', u'weight_1_0', u'weight_1_1',
       u'weight_1_2', u'weight_1_3', u'weight_1_4', u'weight_1_5',
       u'weight_1_6', u'weight_1_7', u'weight_2_0', u'weight_2_1',
       u'weight_2_2', u'weight_2_3', u'weight_2_4', u'weight_2_5',
       u'weight_2_6', u'weight_2_7', u'weight_3_0', u'weight_3_1',
       u'weight_3_2', u'weight_3_3', u'weight_3_4', u'weight_3_5',
       u'weight_3_6', u'weight_3_7', u'weight_4_0', u'weight_4_1',
       u'weight_4_2', u'weight_4_3', u'weight_4_4', u'weight_4_5',
       u'weight_4_6', u'weight_4_7', u'weight_5_0', u'weight_5_1',
       u'weight_5_2', u'weight_5_3', u'weight_5_4', u'weight_5_5',
       u'weight_5_6', u'weight_5_7', u'weight_6_0', u'weight_6_1',
       u'weight_6_2', u'weight_6_3', u'weight_6_4', u'weight_6_5',
       u'weight_6_6', u'weight_6_7', u'weight_7_0', u'weight_7_1',
       u'weight_7_2', u'weight_7_3', u'weight_7_4', u'weight_7_5',
       u'weight_7_6', u'weight_7_7', u'num_holes']

### Image Selection Logic

In [3]:
def on_path_select(b):
    global FILES
    FILES = sorted(glob.glob(os.path.join(input_path.value, "*.png")))
    dropdown_image.options = FILES

def on_image_select(b):
    global postal_code_images
    file_name = dropdown_image.value
    if file_name:
        postal_code_images = get_individual_images(file_name)
        # update UI
        image_preview.value = w.Image.from_file(file_name).value # ipywidgets Image class
        reset_labels()
        update_image_preview(postal_code_images)

def update_image_preview(images):
    if images:
        for i in range(4):
            digit_previews[i].clear_output()
            with digit_previews[i]:
                images[i][1].resize(74, 74).invert().save(disp)

### Image Separation Logic

In [4]:
# Return an array of individual images and their digit from the image object
def get_individual_images(file_name):
    # SimpleCV Image class
    img = Image(str(file_name))
    return [
        get_image_obj(crop(img, 0)), 
        get_image_obj(crop(img, 1)), 
        get_image_obj(crop(img, 2)), 
        get_image_obj(crop(img, 3))
    ]

def get_image_obj(img):
    _bin = img.binarize()
    return (None, _bin.dilate().erode(), _bin)

# Crop and return a part of the source image
def crop(img, part):
    return img.crop(SIZE * part, 0, SIZE, SIZE)

### Feature Extraction Logic

In [5]:
def get_weighted_matrix(img):
    T = 5 # THRESHOLD
    CROP = SIZE / COUNT
    m = np.zeros((COUNT, COUNT))
    for y in range(COUNT):
        for x in range(COUNT):
            part = img.crop(x * CROP, y * CROP, CROP, CROP)
            sum = (part.getNumpy()[:,:,0] / 255).sum()
            m[x][y] = sum if sum > T else 0
    return m.T

def extract_features(image):
    # Columns detected using computer vision
    features_dict = {
        "area": [],
        "width": [],
        "contours": [],
        "radius": [],
        "circle_dist": [],
        "rect_dist": [],
        "hull_radius": [],
        "aspect_ratio": [],
        "centroid_x": [],
        "centroid_y": [],
        "angle": [],
        "corners": [],
        "circles": [],
        "num_holes": [],
    }

    # Reorder columns
    column_order = [
        "area",
        "width",
        "contours",
        "radius",
        "circle_dist",
        "rect_dist",
        "hull_radius",
        "aspect_ratio",
        "centroid_x",
        "centroid_y",
        "corners",
        "circles",
        "angle"
    ]

    # Adding column names for the low-res pixel count matrix
    for x in range(COUNT):
        for y in range(COUNT):
            name = '_'.join(['weight', str(x), str(y)])
            column_order.append(name)
            features_dict[name] = []

    column_order.append("num_holes")
    
    img = image[1]
    raw = image[2]
    img.show()
    
    # Find blobs in the preprocessed image
    blobs = img.findBlobs()
    if blobs:
        # Fractured digits
        if len(blobs) > 1:
            # Try again with more dilation
            img = img.dilate()
            blobs = img.findBlobs()
            if len(blobs) > 1:
                # Try again with raw image
                img = raw
                blobs = img.findBlobs()
        
        blob = blobs[0]
        features_dict['area'].append(blob.area())
        features_dict['width'].append(blob.width())
        features_dict['contours'].append(len(blob.contour()))
        features_dict['circle_dist'].append(blob.circleDistance())
        features_dict['rect_dist'].append(blob.rectangleDistance())
        features_dict['radius'].append(blob.radius())
        features_dict['hull_radius'].append(blob.hullRadius())
        features_dict['aspect_ratio'].append(blob.aspectRatio())
        features_dict['centroid_x'].append(blob.centroid()[0])
        features_dict['centroid_y'].append(blob.centroid()[1])
        features_dict['angle'].append(blob.angle())
        features_dict['corners'].append(len(img.findCorners()))
        
        # Finding the number of circles in an image
        circles_set = img.embiggen(2).scale(4).findCircle(thresh=40, distance=20)
        circles = 0
        if circles_set:
            circles = len(circles_set)
        features_dict['circles'].append(circles)

        # Calculate pixel count matrix in the preprocessed image
        m = get_weighted_matrix(img)
        for x in range(COUNT):
            for y in range(COUNT):
                features_dict['_'.join(['weight', str(x), str(y)])].append(m[x, y])

        # Finding the number of holes in the 'just binarized' image
        num_holes = len(raw.embiggen(2).invert().findBlobs(minsize=3)[:-1])
        features_dict['num_holes'].append(num_holes)
    
    return pd.DataFrame(features_dict, columns=column_order)

### Prediction Logic

In [6]:
def predict(img):
    # extract features
    features = extract_features(img)[COLUMNS]
    if len(features) == 1:
        # normalize all features
        features = pd.DataFrame(SCALER.transform(features), columns=features.columns)
        # predict
        return CLASSIFIER.predict(features)[0]
    return None

def predict_all(images):
    return [predict(images[0]), predict(images[1]), predict(images[2]), predict(images[3])]

def predict_dir(b):
    output_predict_dir.clear_output()
    for file_name in FILES:
        images = get_individual_images(file_name)
        with output_predict_dir:
            ps = predict_all(images)
            postal_code = str(ps[0]) + ' ' + str(ps[1]) + ' ' + str(ps[2]) + ' ' + str(ps[3])
            image = w.Image.from_file(file_name)
            image.layout.width = '20%'
            image.layout.height = '20%'
            display(HBox([image, w.Label(value=postal_code)], layout={'margin': '0'}))

### Image Selection UI

In [7]:
input_path = w.Text(value="../dataset-images/", placeholder="path", description="Directory:")
button_select_path = w.Button(description='Use')
button_select_path.on_click(on_path_select)

dropdown_image = w.Dropdown(description='Image:')
button_select_image = w.Button(description="Use")
button_select_image.on_click(on_image_select)
image_preview = w.Image(layout={'width': '340px'})

image_selection = VBox([
    HBox([input_path, button_select_path]),
    HBox([dropdown_image, button_select_image])
])

### Prediction UI

In [8]:
digit_previews = [w.Output(), w.Output(), w.Output(), w.Output()]

layout_predict_button = {'width': '82px'}
buttons_predict = [
    w.Button(description="Predict", layout=layout_predict_button),
    w.Button(description="Predict", layout=layout_predict_button),
    w.Button(description="Predict", layout=layout_predict_button),
    w.Button(description="Predict", layout=layout_predict_button)
]

layout_predict_label = {'margin': '0 auto'}
labels_predict = [
    w.Label(layout=layout_predict_label),
    w.Label(layout=layout_predict_label),
    w.Label(layout=layout_predict_label),
    w.Label(layout=layout_predict_label)
]

def reset_labels(value='-'):
    for i in range(4):
        set_label(i, value)

def set_label(index, value):
    labels_predict[index].value = str(value)

def predict_all_ui(b):
    predictions = predict_all(postal_code_images)
    for i in range(4):
        set_label(i, predictions[i])

# Setup listeners
buttons_predict[0].on_click(lambda b: set_label(0, predict(postal_code_images[0])))
buttons_predict[1].on_click(lambda b: set_label(1, predict(postal_code_images[1])))
buttons_predict[2].on_click(lambda b: set_label(2, predict(postal_code_images[2])))
buttons_predict[3].on_click(lambda b: set_label(3, predict(postal_code_images[3])))

button_predict_all = w.Button(description="Predict Postal Code", layout={'width': '340px'})
button_predict_all.on_click(predict_all_ui)

button_predict_dir = w.Button(description="Predict All Images")
button_predict_dir.on_click(predict_dir)
output_predict_dir = w.Output()#layout={'height': '400px', 'overflow-y': 'scroll'})

selection = HBox([
    VBox([
        image_selection,
        image_preview,
        HBox([
            VBox([digit_previews[0], labels_predict[0], buttons_predict[0]]),
            VBox([digit_previews[1], labels_predict[1], buttons_predict[1]]),
            VBox([digit_previews[2], labels_predict[2], buttons_predict[2]]),
            VBox([digit_previews[3], labels_predict[3], buttons_predict[3]])
        ]),
        button_predict_all
    ], layout={'width': '360px'}),
    VBox([
        button_predict_dir,
        w.Label(value="All image predictions:"),
        output_predict_dir
    ], layout={'margin': '0 0 0 64px', 'width': '360px'})
])
    
with app:
    display(selection)
    
    # Initial situation
    on_path_select(None)
    on_image_select(None)

# Digit Recognition App

### How to use:

#### Predict a single postal code

1. Select a folder path.
2. Select a 128x32 pixel 'postal code' image to recognize.
3. Predict individual digits or the whole postal code at once.

#### Predict all postal codes

1. Select a folder path.
2. Predict all images in the folder.

In [9]:
app

Output()